In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/Colab\ Notebooks

/content/gdrive/MyDrive/Colab Notebooks


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import evaluate  # Import the new evaluate library
from sklearn.metrics import accuracy_score, f1_score

# Check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load the GLUE dataset (SST-2 for sentiment analysis in this example)
glue = load_dataset("glue", "sst2")

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
# Tokenizer and model setup
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True, padding="max_length")

# Tokenize the GLUE dataset
tokenized_glue = glue.map(tokenize_function, batched=True)



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
# Custom FFN layer with ReLU, GLU, and GLU variants
class BertWithCustomFFN(BertForSequenceClassification):
    def __init__(self, config, activation='relu'):
        super().__init__(config)
        self.activation = activation
        self.hidden_size = config.hidden_size

        # Define weight matrices for GLU variants
        self.W = nn.Linear(self.hidden_size, self.hidden_size * 2)  # First linear transformation
        self.V = nn.Linear(self.hidden_size, self.hidden_size * 2)  # Second linear transformation
        self.W2 = nn.Linear(self.hidden_size * 2, self.hidden_size)  # Final output linear transformation

    # Custom FFN layer with GLU variants
    def custom_ffn(self, x):
        a = self.W(x)  # Linear transformation 1
        b = self.V(x)  # Linear transformation 2

        # Apply gating mechanism based on activation
        if self.activation == 'relu':
            h_activated = F.relu(a)
        elif self.activation == 'glu':
            h_activated = torch.sigmoid(a) * b  # GLU
        elif self.activation == 'reglu':
            h_activated = F.relu(a) * b  # ReGLU
        elif self.activation == 'geglu':
            h_activated = 0.5 * a * (1 + torch.tanh((torch.sqrt(torch.tensor(2 / torch.pi)) * (a + 0.044715 * a**3)))) * b  # GEGLU
        elif self.activation == 'swiglu':
            h_activated = a * torch.sigmoid(a) * b  # SwiGLU
        else:
            raise ValueError(f"Unknown activation function: {self.activation}")

        out = self.W2(h_activated)  # Apply the second linear transformation
        return out

    # Forward pass of BERT with custom FFN
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                inputs_embeds=None, labels=None, output_hidden_states=True):
        # Call the default BERT forward function
        outputs = super().forward(input_ids, attention_mask, token_type_ids, position_ids, head_mask, inputs_embeds, labels, output_hidden_states=output_hidden_states)

        # Ensure hidden_states is returned
        hidden_states = outputs.hidden_states[-1]  # Use the last hidden layer of BERT
        hidden_states = self.custom_ffn(hidden_states)

        return outputs

# Initialize models with different activation functions (ReLU, GLU, ReGLU, GEGLU, SwiGLU)
model_relu = BertWithCustomFFN.from_pretrained("bert-base-uncased", num_labels=2, activation='relu').to(device)
model_glu = BertWithCustomFFN.from_pretrained("bert-base-uncased", num_labels=2, activation='glu').to(device)
model_reglu = BertWithCustomFFN.from_pretrained("bert-base-uncased", num_labels=2, activation='reglu').to(device)
model_geglu = BertWithCustomFFN.from_pretrained("bert-base-uncased", num_labels=2, activation='geglu').to(device)
model_swiglu = BertWithCustomFFN.from_pretrained("bert-base-uncased", num_labels=2, activation='swiglu').to(device)

# Define data collator for training and validation datasets (no need to move to device here)
data_collator = lambda x: {
    'input_ids': torch.tensor([f['input_ids'] for f in x]),
    'attention_mask': torch.tensor([f['attention_mask'] for f in x]),
    'labels': torch.tensor([f['label'] for f in x]),
}

# Load the evaluation metric using the new 'evaluate' library
metric = evaluate.load("glue", "sst2")

# Define the function to compute evaluation metrics (Accuracy, F1 Score)
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": accuracy, "f1": f1}

# Training arguments with W&B disabled
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none"  # Disable W&B logging
)

# Fine-tuning the models with different activations (ReLU, GLU, ReGLU, GEGLU, SwiGLU)
trainer_relu = Trainer(
    model=model_relu,
    args=training_args,
    train_dataset=tokenized_glue['train'],
    eval_dataset=tokenized_glue['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer_glu = Trainer(
    model=model_glu,
    args=training_args,
    train_dataset=tokenized_glue['train'],
    eval_dataset=tokenized_glue['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer_reglu = Trainer(
    model=model_reglu,
    args=training_args,
    train_dataset=tokenized_glue['train'],
    eval_dataset=tokenized_glue['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer_geglu = Trainer(
    model=model_geglu,
    args=training_args,
    train_dataset=tokenized_glue['train'],
    eval_dataset=tokenized_glue['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer_swiglu = Trainer(
    model=model_swiglu,
    args=training_args,
    train_dataset=tokenized_glue['train'],
    eval_dataset=tokenized_glue['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertWithCustomFFN were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['V.bias', 'V.weight', 'W.bias', 'W.weight', 'W2.bias', 'W2.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertWithCustomFFN were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['V.bias', 'V.weight', 'W.bias', 'W.weight', 'W2.bias', 'W2.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertWithCustomFFN were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['V.bias', 'V.weight', 'W.bias', 'W.weight', 'W2.bias', 'W2.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train models
trainer_relu.train()
trainer_glu.train()
#trainer_reglu.train()
#trainer_geglu.train()
#trainer_swiglu.train()

Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 444.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 349.06 MiB is free. Process 21105 has 14.40 GiB memory in use. Of the allocated memory 13.83 GiB is allocated by PyTorch, and 458.18 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Evaluate models
eval_results_relu = trainer_relu.evaluate()
eval_results_glu = trainer_glu.evaluate()
eval_results_reglu = trainer_reglu.evaluate()
eval_results_geglu = trainer_geglu.evaluate()
eval_results_swiglu = trainer_swiglu.evaluate()

# Print evaluation results (Accuracy, F1 Score, etc.)
print("Evaluation results with ReLU activation:", eval_results_relu)
print("Evaluation results with GLU activation:", eval_results_glu)
print("Evaluation results with ReGLU activation:", eval_results_reglu)
print("Evaluation results with GEGLU activation:", eval_results_geglu)
print("Evaluation results with SwiGLU activation:", eval_results_swiglu)
